##### ARTI 560 - Computer Vision  
## Image Classification using Transfer Learning - Exercise 

### Objective

In this exercise, you will:

1. Select another pretrained model (e.g., VGG16, MobileNetV2, or EfficientNet) and fine-tune it for CIFAR-10 classification.  
You'll find the pretrained models in [Tensorflow Keras Applications Module](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

2. Before training, inspect the architecture using model.summary() and observe:
- Network depth
- Number of parameters
- Trainable vs Frozen layers

3. Then compare its performance with ResNet and the custom CNN.

### Questions:

- Which model achieved the highest accuracy?
- Which model trained faster?
- How might the architecture explain the differences?

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess

# 1. Define Data Augmentation 
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augmentation")

# 2. Build the base model (MobileNetV2)
mobilenet_base = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

# 3. Freeze the backbone initially
mobilenet_base.trainable = False 

# 4. Construct the full model
mobilenet_model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation, 
    layers.Resizing(224, 224, interpolation="bilinear"),
    layers.Lambda(mobilenet_preprocess), 
    mobilenet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10) 
], name="cifar10_mobilenetV2")

# 5. Inspect the architecture
# This will show the depth, parameters, and frozen status
mobilenet_model.summary()

Model: "cifar10_mobilenetV2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
import tensorflow as tf
from tensorflow import keras

# 1. Reload the data because the session lost it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32")
x_test  = x_test.astype("float32")
y_train = y_train.squeeze().astype("int64")
y_test  = y_test.squeeze().astype("int64")

# 2. Unfreeze the last 20 layers of MobileNetV2
mobilenet_base.trainable = True
for layer in mobilenet_base.layers[:-20]:
    layer.trainable = False

print(f"Trainable layers in MobileNet: {sum(l.trainable for l in mobilenet_base.layers)}")

# 3. Compile the model
mobilenet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# 4. Train the model (This will now find x_train and y_train)
history_mobile = mobilenet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    verbose=1
)

# 5. Evaluate on test data
test_loss_m, test_acc_m = mobilenet_model.evaluate(x_test, y_test, verbose=0)
print(f"\nMobileNetV2 Fine-tuned Test Accuracy: {test_acc_m}")

c:\Users\DELL\anaconda3\envs\cv_lab\Lib\site-packages\keras\src\datasets\cifar.py:18: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


Trainable layers in MobileNet: 20
Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 1848s 3s/step - accuracy: 0.5566 - loss: 1.3164 - val_accuracy: 0.7376 - val_loss: 0.7674
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 1633s 2s/step - accuracy: 0.7289 - loss: 0.7977 - val_accuracy: 0.8136 - val_loss: 0.5408
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 1727s 2s/step - accuracy: 0.7672 - loss: 0.6812 - val_accuracy: 0.8340 - val_loss: 0.4701

MobileNetV2 Fine-tuned Test Accuracy: 0.8327999711036682


In [ ]:
# Final Performance Comparison 
print("\n--- Final Performance Comparison ---")

# 1. Custom CNN: From Lab 1 results
custom_cnn_acc = 0.7099
print(f"1. Custom CNN Accuracy:  {custom_cnn_acc:.4f}")

# 2. ResNet50V2: From Lab 2 results
resnet_acc = 0.9161
print(f"2. ResNet50V2 Accuracy:  {resnet_acc:.4f}")

# 3. MobileNetV2: This Exercise run
mobilenet_acc = 0.8328
print(f"3. MobileNetV2 Accuracy: {mobilenet_acc:.4f}")

print("\n--- Model Architecture Context ---")
print(f"- MobileNetV2 Total Params: 2,270,794")
print(f"- ResNet50V2 Total Params:  ~23,500,000")

print("\n--- Exercise Insights ---")
print(f"- Accuracy Gap: Transfer Learning (ResNet/MobileNet) outperformed Custom CNN by over 12-20%.")
print(f"- Efficiency: MobileNetV2 achieved 83.28% accuracy with only 10% of ResNet's parameters.")


--- Final Performance Comparison ---
1. Custom CNN Accuracy:  0.7099
2. ResNet50V2 Accuracy:  0.9161
3. MobileNetV2 Accuracy: 0.8328

--- Model Architecture Context ---
- MobileNetV2 Total Params: 2,270,794
- ResNet50V2 Total Params:  ~23,500,000

--- Exercise Insights ---
- Accuracy Gap: Transfer Learning (ResNet/MobileNet) outperformed Custom CNN by over 12-20%.
- Efficiency: MobileNetV2 achieved 83.28% accuracy with only 10% of ResNet's parameters.


Exercise Discussion & Comparison

1- Which model achieved the highest accuracy?
The ResNet50V2 model achieved the highest accuracy at 91.62%. While MobileNetV2 performed impressively well with 83.28% in only 3 epochs, ResNet’s deeper architecture allows it to capture more complex features, leading to superior overall precision.

2- Which model trained faster?
MobileNetV2 was significantly faster during training. The training process was smooth and efficient, which is directly attributed to the much smaller number of parameters the system had to process compared to the other models.

3- How might the architecture explain the differences?
The differences in performance and speed are fundamentally rooted in the architectural design of each network:

ResNet50V2: This is a massive architecture with approximately 23.5 million parameters. Its depth makes it highly accurate but also computationally "heavy" and slower to train.

MobileNetV2: Designed for efficiency on mobile and embedded devices, it contains only about 2.27 million total parameters. It uses Depthwise Separable Convolutions, which drastically reduces the mathematical operations required while maintaining high accuracy.

Custom CNN: This basic model had the lowest accuracy (70.99%) because it lacks the "pre-trained knowledge" that Transfer Learning models (ResNet and MobileNet) gain from being trained on the massive ImageNet dataset.